In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec.load("/kaggle/input/transfer/300features_40minwords_10context")

In [ ]:
type(model.wv["queen"])

In [ ]:
len(model.wv)

In [ ]:
num_features = len(model.wv['queen'])

In [ ]:
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype='float32')
    nwords = 0
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            nwords += 1
            featureVec = np.add(featureVec, model.wv[word])
            
    featureVec = np.divide(featureVec, nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")
    for review in reviews:
        if counter % 1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter += 1
    return reviewFeatureVecs

In [ ]:
train = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip",
                    header=0, delimiter="\t", quoting=3)
test = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip",
                   header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip",
                              header=0, delimiter="\t", quoting=3)

In [ ]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [ ]:
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)
print("Creating average feature vectors for test reviews")
    
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100)

print("Fitting a random forest to labeled training data...")
forest = forest.fit(trainDataVecs, train["sentiment"])

result = forest.predict(testDataVecs)

output = pd.DataFrame(data={"id": test["id"], "sentiment": result})
output.to_csv("Word2Vec_AverageVectors.csv", index=False, quoting=3)

In [ ]:
from sklearn.cluster import KMeans
import time

start = time.time()

word_vectors = model.wv.vectors
num_clusters = 3298 # word_vectors.shape[0] / 5

kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

end = time.time()
elapsed = end-start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

In [ ]:
word_centroid_map = dict(zip(model.wv.index_to_key, idx))

In [ ]:
type(word_centroid_map.values())

In [ ]:
word_centroid_map_values = list(word_centroid_map.values())
word_centroid_map_keys = list(word_centroid_map.keys())

for cluster in np.arange(0, 3):
    print("\nCluster %d" % cluster)
    words = []
    for i in np.arange(0, len(word_centroid_map_values)):
        if(word_centroid_map_values[i] == cluster):
            words.append(word_centroid_map_keys[i])
    print(words)

In [ ]:
def create_bag_of_centroids(wordlist, word_centroid_map):
    num_centroids = max(word_centroid_map.values()) + 1
    bag_of_centroids = np.zeros(num_centroids, dtype='float32')
    
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
            
    return bag_of_centroids

In [ ]:
train_centroids = np.zeros((train['review'].size, num_clusters), dtype='float32')

counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1
    
test_centroids = np.zeros((test['review'].size, num_clusters), dtype='float32')

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

In [ ]:
forest = RandomForestClassifier(n_estimators=100)
print("Fitting a random forest to labeled training data...")

forest = forest.fit(train_centroids, train["sentiment"])
result = forest.predict(test_centroids)

output = pd.DataFrame(data={'id': test['id'], 'sentiment': result})
output.to_csv('BagOfCentroids.csv', index=False, quoting=3)